In [1]:
# set up
import pandas as pd
import numpy as np
import pickle
import time 
import os
import matplotlib.pyplot as plt
import datetime
from operator import itemgetter
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pylab import *
from matplotlib.font_manager import FontProperties  
data_folder_path = 'C:\\Users\\goodluck\\Desktop\\DB'
raw_data_path =  "C:\\Users\\goodluck\\Desktop\\DB\\raw_data"
storage_path =  "C:\\Users\\goodluck\\Desktop\\DB\\database_storage"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
font = FontProperties(fname=r"simsun.ttf", size=14)  
matplotlib.rcParams['axes.unicode_minus'] =False
mpl.rcParams['font.sans-serif'] = ['SimHei']
tick_spacing = 4

# re-open
with open(DTBS_path, 'rb') as f:  
    DTBS = pickle.load(f)
    
def takeSecond(elem):
    return elem[1]


def retrieve_low_nodes(codes,dt,num):
    dl_rank = []
    for code in codes:
        if DTBS['A'][code][dt]['dl'] != 999:
            dl_rank.append((code, DTBS['A'][code][dt]['dl']))
    
    dl_rank.sort(key=takeSecond)
    low = dl_rank[0:num]
    return [i[0] for i in low]

def past_qs(days, code):
    
    for day in days:
            
        if DTBS['A'][code][day]['qs'] == 1:
            return 1
    
    return 0

def select_condition(codes, dt, fcd):
    satisfy = []
    if fcd[0] == 'lp':
        lm = calculate_time(dt, 'p', 20)
        
        for code in codes:
            if low_prc(code, dt, lm, fcd[1]):
                satisfy.append(code)
        
    else:
        for code in codes:
            if DTBS['A'][code][dt][fcd[0]] <= fcd[2] and DTBS['A'][code][dt][fcd[0]] >= fcd[1]:
                satisfy.append(code)
                    
    return satisfy

def rank(codes, dt, idx, num, d):
    rk = []
    if num == -1:
        num = len(codes)
    for code in codes:
        if idx == 'lp':
            if not np.isnan(DTBS['A'][code][dt]['dp']):
                lm = calculate_time(dt, 'p', 20)
                prc = get_prcs(code, lm)
                lp = absolute_low(prc, 0)
                rk.append((code, DTBS['A'][code][dt]['dp']/lp))
            
            
        elif idx != 'dl':
            if not np.isnan(DTBS['A'][code][dt][idx]):
                rk.append((code, DTBS['A'][code][dt][idx]))
        else:
            if not np.isnan(DTBS['A'][code][dt][idx]):
                if DTBS['A'][code][dt][idx] != 999:
                    rk.append((code, DTBS['A'][code][dt][idx]))
    if num >= len(rk):
        num = len(rk)
        
     
#     reverse = True 降序
    if d == 'd':
        rk.sort(key=takeSecond, reverse = True)
    else:
        rk.sort(key=takeSecond, reverse = False)

    if idx == 'lp':
        lp = [i for i in rk if i[1] == 1.0]
        ot = []
        if num - len(lp) > 0:
            ot = rk[len(lp):num]
        
        low = lp + ot
        
    else:    
        low = rk[0:num]
#     print("In",dt , len(low), "of", len(codes),  "is selected")   

#     print(low)
    
    
    return [i[0] for i in low]
  

def get_prcs(code, dts):
    prc = []
    for dt in dts:
        prc.append(DTBS['A'][code][dt]['dp'])
    prc.sort()
    return prc
    
def relative_low(lst, pos):
    return np.quantile(a=lst, q = pos)
    
def absolute_low(lst, pos):
    return min(lst) + pos * (max(lst) - min(lst))

def low_prc(code, dt, lm, pos):
    prc = get_prcs(code, lm)
#     lp = min([relative_low(prc, pos), absolute_low(prc, pos)])
    lp = absolute_low(prc, pos)
    tp = DTBS['A'][code][dt]['dp']
#     if (tp <= lp):
#         print(code, tp, lp)
#     if tp <= lp:
#         print(tp, lp)
    return tp <= lp

def calculate_time(dt, d, l):
    time = []
    days = DTBS['D']['day']
    if d == 'p':
        for i in range(len(days)):
            if days[i] == dt:
                break
                
    return days[i-l+1:i+1]

past_dt = DTBS['D']['day'][-20:]

In [2]:
dt = DTBS['D']['day'][-1]
print(dt)
dt = DTBS['D']['day'][-1]
past_dt = DTBS['D']['day'][-20:]
re = retrieve_low_nodes(DTBS['A'].keys(), dt , 400)


2023-05-22


In [6]:
# # save
# f_save = open(DTBS_path, 'wb')
# pickle.dump(DTBS, f_save)
# f_save.close()


In [7]:

print(DTBS['A']['110077.SH']['2023-05-04'])
print(DTBS['B']['110077.SH'])
print(DTBS['E'][DTBS['B']['110077.SH']['sc']]['2023-05-04'])



{'cpr': 1.09, 'dp': 133.001, 'dl': 134.091, 'ytm': -4.57, 'bl': 17.672, 'trt': 9.77, 'yl': 3.548, 'csp': 6.21, 'ia': 1, 'qs': 0, 'xx': 0, 'hs': 0, 'csv': 131.5619967793881, 'cz': 101.0871, 'pd': 0.30147166927716884}
{'cn': '洪城转债', 'ipo': '2020-12-17', 'sc': '600461.SH', 'sn': '洪城环境', 'cat1': '环保(2021)', 'cat2': '环境治理(2021)', 'dld': '2046-01-01'}
{'cl': 8.17, 'pb': 1.2}


In [8]:
6.21 * 1.3

8.073

In [5]:
dt = DTBS['D']['day'][-1]
# dt = '2023-01-30'
# dt = '2023-04-12'
select = list(DTBS['A'].keys())
past_dt = DTBS['D']['day'][-40:]

# flt_cd = [['lp', 0.5], ['dp', 110, 130], ['bl',0, 7], ['cpr',-999, 50]]

flt_cd = [['dp', 100, 130]]

flt_cd = [['dp', 100, 130], ['bl',0, 3]]
flt_cd = [['lp', 0.5], ['dp', 100, 130], ['bl',0, 3], ['cpr', -999, 50]]
flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]
flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', 0.5, 9999]]

flt_cd = []
flt_cd = [['lp', 0.5], ['dp', 100, 130], ['bl',0, 3], ['cpr', -999, 50]]
for fcd in flt_cd:
    select = select_condition(select, dt, fcd)
    
    
# rk_cd = [['lp', 10, 'a']]
rk_cd = [['dl', 600, 'a']]
rk_cd = [['ytm', 20, 'd']]
rk_cd = [['dl', 20, 'a']]
    

rk_cd = [['dl', 400, 'a']]
rk_cd = [['lp', 20, 'a']]
lp = rank(select, dt, rk_cd[0][0], rk_cd[0][1], rk_cd[0][2])
print(len(lp))
print("当前日期：", dt)
for r in lp:
    print(r, DTBS['B'][r]['cn'], DTBS['B'][r]['sc'],  DTBS['B'][r]['sn'],  '强赎：' , past_qs(past_dt, r), "收盘价：", DTBS['A'][r][dt]['dp'],  "转股溢价率：", DTBS['A'][r][dt]['cpr'], "到期收益率：", DTBS['A'][r][dt]['ytm'],"换手率：", DTBS['A'][r][dt]['trt'], '市值',  DTBS['A'][r][dt]['bl'], '双低', DTBS['A'][r][dt]['dl'], '申万一级', DTBS['B'][r]['cat1'],'申万二级', DTBS['B'][r]['cat2'])

13
当前日期： 2023-05-22
113526.SH 联泰转债 603797.SH 联泰环保 强赎： 0 收盘价： 126.638 转股溢价率： 24.04 到期收益率： -9.26 换手率： 4.17 市值 1.927 双低 150.678 申万一级 环保(2021) 申万二级 环境治理(2021)
113595.SH 花王转债 603007.SH ST花王 强赎： 0 收盘价： 110.225 转股溢价率： 11.22 到期收益率： 3.88 换手率： 14.0 市值 2.707 双低 121.445 申万一级 建筑装饰(2021) 申万二级 基础建设(2021)
113561.SH 正裕转债 603089.SH 正裕工业 强赎： 0 收盘价： 118.948 转股溢价率： 40.82 到期收益率： -0.98 换手率： 3.24 市值 1.499 双低 159.768 申万一级 汽车(2021) 申万二级 汽车零部件(2021)
128039.SZ 三力转债 002224.SZ 三力士 强赎： 0 收盘价： 115.023 转股溢价率： 43.65 到期收益率： -6.34 换手率： 3.06 市值 2.024 双低 158.673 申万一级 基础化工(2021) 申万二级 橡胶(2021)
128021.SZ 兄弟转债 002562.SZ 兄弟科技 强赎： 0 收盘价： 112.09 转股溢价率： 28.79 到期收益率： -10.43 换手率： 3.37 市值 2.651 双低 140.88 申万一级 基础化工(2021) 申万二级 化学制品(2021)
113566.SH 翔港转债 603499.SH 翔港科技 强赎： 0 收盘价： 120.801 转股溢价率： 38.41 到期收益率： 0.36 换手率： 1.54 市值 1.613 双低 159.211 申万一级 轻工制造(2021) 申万二级 包装印刷(2021)
128053.SZ 尚荣转债 002551.SZ 尚荣医疗 强赎： 0 收盘价： 126.8 转股溢价率： 47.33 到期收益率： -7.05 换手率： 3.33 市值 1.898 双低 174.13 申万一级 医药生物(2021) 申万二级 医疗器械(2021)
128056.SZ 今飞转债 002863.SZ 今飞凯达 强赎：

In [4]:
dt = DTBS['D']['day'][-1]
# dt = '2023-04-12'
select = list(DTBS['A'].keys())

# flt_cd = [['lp', 0.5], ['dp', 110, 130], ['bl',0, 7], ['cpr',-999, 50]]
flt_cd = [['lp', 0.5], ['dp', 100, 130], ['bl',0, 3], ['cpr', -999, 50]]
# flt_cd = [['dp', 100, 130]]
# flt_cd = [['dp', 100, 130], ['bl',0, 3]]
for fcd in flt_cd:
    select = select_condition(select, dt, fcd)
    
    
rk_cd = [['lp', 10, 'a']]
# rk_cd = [['dl', 600, 'a']]
# rk_cd = [['dl', 10, 'a']]
lp = rank(select, dt, rk_cd[0][0], rk_cd[0][1], rk_cd[0][2])
len(lp)
print("当前日期：", dt)
for r in lp:
    print(r, DTBS['B'][r]['cn'], DTBS['B'][r]['sc'],  DTBS['B'][r]['sn'],  '强赎：' , past_qs(past_dt, r), "收盘价：", DTBS['A'][r][dt]['dp'],  "转股溢价率：", DTBS['A'][r][dt]['cpr'], "到期收益率：", DTBS['A'][r][dt]['ytm'],"换手率：", DTBS['A'][r][dt]['trt'], '市值',  DTBS['A'][r][dt]['bl'], '双低', DTBS['A'][r][dt]['dl'], '申万一级', DTBS['B'][r]['cat1'],'申万二级', DTBS['B'][r]['cat2'])

当前日期： 2023-05-12
113566.SH 翔港转债 603499.SH 翔港科技 强赎： 0 收盘价： 119.385 转股溢价率： 38.26 到期收益率： 0.79 换手率： 5.1 市值 1.613 双低 157.645 申万一级 轻工制造(2021) 申万二级 包装印刷(2021)
123152.SZ 润禾转债 300727.SZ 润禾材料 强赎： 0 收盘价： 121.0 转股溢价率： 36.22 到期收益率： 0.56 换手率： 12.09 市值 2.92 双低 157.22 申万一级 基础化工(2021) 申万二级 化学制品(2021)
123166.SZ 蒙泰转债 300876.SZ 蒙泰高新 强赎： 0 收盘价： 122.536 转股溢价率： 29.79 到期收益率： -0.16 换手率： 3.22 市值 3.0 双低 152.326 申万一级 基础化工(2021) 申万二级 化学纤维(2021)
128039.SZ 三力转债 002224.SZ 三力士 强赎： 0 收盘价： 115.058 转股溢价率： 45.27 到期收益率： -6.21 换手率： 3.87 市值 2.024 双低 160.328 申万一级 基础化工(2021) 申万二级 橡胶(2021)
128071.SZ 合兴转债 002228.SZ 合兴包装 强赎： 0 收盘价： 112.396 转股溢价率： 38.86 到期收益率： 0.36 换手率： 3.7 市值 2.999 双低 151.256 申万一级 轻工制造(2021) 申万二级 包装印刷(2021)
113030.SH 东风转债 601515.SH 东风股份 强赎： 0 收盘价： 118.424 转股溢价率： 33.5 到期收益率： -1.03 换手率： 5.48 市值 2.945 双低 151.924 申万一级 轻工制造(2021) 申万二级 包装印刷(2021)
113526.SH 联泰转债 603797.SH 联泰环保 强赎： 0 收盘价： 125.79 转股溢价率： 23.21 到期收益率： -8.75 换手率： 5.69 市值 1.927 双低 149.0 申万一级 环保(2021) 申万二级 环境治理(2021)
123061.SZ 航新转债 300424.SZ 航新科技 强赎： 0 收盘价： 12

In [5]:
dt = DTBS['D']['day'][-1]
dt

'2023-04-11'

In [6]:
lm =  DTBS['D']['day'][-20:]

In [4]:
(196.96 - 197.9855) / 197.9855

-0.0051796722487252546

In [61]:

# code = '118016.SH'
# prc = []
# for dt in lm:
#     prc.append(DTBS['A'][code][dt]['dp'])
# prc.sort()    
# max_prc = max(prc)
# min_prc = min(prc)
# print(prc)
# print(max_prc, min_prc)

def get_prcs(code, dts):
    prc = []
    for dt in dts:
        prc.append(DTBS['A'][code][dt]['dp'])
    prc.sort()
    return prc
    
def relative_low(lst, pos):
    return np.quantile(a=lst, q = pos)
    
def absolute_low(lst, pos):
    return min(lst) + pos * (max(lst) - min(lst))

def low_prc(code, dt, lm, pos):
    prc = get_prcs(code, lm)
#     lp = min([relative_low(prc, pos), absolute_low(prc, pos)])
    lp = absolute_low(prc, pos)
    tp = DTBS['A'][code][dt]['dp']
#     print(code, tp, lp, tp <= lp)
#     if tp <= lp:
#         print(tp, lp)
    return tp <= lp

In [62]:
113600.SH

SyntaxError: invalid decimal literal (3743447309.py, line 1)

In [63]:
max([1,2,3])

3

In [64]:
low_prc('123128.SZ', DTBS['D']['day'][-1], DTBS['D']['day'][-20:], 0.2)


123128.SZ 102.006 100.96039999999999 False


False

In [65]:
print(min(prc))
print(np.quantile(a=prc, q=1))

NameError: name 'prc' is not defined

In [66]:


relative_low(prc, 0.20)
absolute_low(prc, 0.20)

NameError: name 'prc' is not defined

In [25]:
lm =  DTBS['D']['day'][-22:]
dt = DTBS['D']['day'][-1]
pos = 0
low_code = []
for code in DTBS['A'].keys():
    flag = 0
    for odt in lm:
        if DTBS['A'][code][odt]['ia'] == 0:
            flag = 1
    if flag == 0:
        if low_prc(code, dt, lm, pos):
            low_code.append(code)
    

print(len(low_code))

lowlow = []
for code in low_code:
    if DTBS['A'][code][dt]['bl'] <3:
        lowlow.append(code)
        
print(len(lowlow))
print(lowlow)

87
31
['113566.SH', '123012.SZ', '123013.SZ', '123118.SZ', '128039.SZ', '113030.SH', '113574.SH', '123014.SZ', '123034.SZ', '123067.SZ', '123080.SZ', '123159.SZ', '128070.SZ', '128076.SZ', '128120.SZ', '128143.SZ', '113537.SH', '113575.SH', '123011.SZ', '123072.SZ', '127004.SZ', '128056.SZ', '128082.SZ', '113565.SH', '123018.SZ', '128021.SZ', '128025.SZ', '128053.SZ', '113573.SH', '127080.SZ', '128033.SZ']


In [49]:
print(low_code)

['110088.SH', '113631.SH', '123128.SZ', '113046.SH', '113535.SH', '113623.SH', '113641.SH', '118000.SH', '118005.SH', '118010.SH']


In [30]:
DTBS['A'][code][dt]['dp']

117.89200000000001

In [40]:
DTBS['A']['113600.SH'][DTBS['D']['day'][-1]]['dp']

119.941

In [88]:



low_prc('128105.SZ', DTBS['D']['day'][-1], DTBS['D']['day'][-20:], 0.2)



128105.SZ 106.208 106.4964 106.6056 106.4964


True

In [22]:
dt = DTBS['D']['day'][-1]
can = []
for code in DTBS['A'].keys():

    if DTBS['A'][code][dt]['ia'] == 1:
#         print(DTBS['A'][code][dt]['bl'])
        if DTBS['A'][code][dt]['bl'] <= 7:
            if DTBS['A'][code][dt]['trt'] >= 2:
                if DTBS['A'][code][dt]['trt'] <= 10:
                    if DTBS['A'][code][dt]['ytm'] >= -1:
                        can.append(code)

print(len(can))
re = retrieve_low_nodes(can, dt , 400)

26


In [ ]:
%matplotlib notebook